In [1]:
# 周期境界、Plotly版、sliderを使ってmodeをアニメーション
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

## パラメータ設定 ##　格子定数は1に設定
m0 = 1            # 基準の質量
k0 = 1            # 基準のバネ定数
m_num = 2       # 単位格子内の質点の数
k_num = 33        # 波数を変化させる数

k_array = np.linspace(-1, 1, k_num)*np.pi  # 波数の配列
omega_square = np.zeros((k_num, m_num))   # 周波数の２乗の配列を準備
eigen_values = np.zeros((k_num, m_num), dtype = np.complex128)    # 固有値の配列を準備、複素数
eigen_vectors = np.zeros((k_num, m_num, m_num), dtype = np.complex128)   # 固有ベクトルの配列を準備、複素数

delta_m = 1/3  # 単位格子内の質量
delta_k = 0    # 単位格子内のバネ定数
m1 = 1/(1 - delta_m)  # 質量1
m2 = 1/(1 + delta_m)  # 質量2
k1 = 1 + delta_k  # バネ1
k2 = 1 - delta_k  # バネ2
m_list = [m1,m2]  # 質量を並べたリスト
k_list = [k1,k2]  # バネ定数を並べたリスト、要素数は質量リストよりもひとつ多い
N = len(m_list)  # 質量の配列から行列サイズNを求める

## 質量の対角行列Mを定義 ##
M = np.zeros((N, N), dtype = np.complex128)
M[0,0] = m1
M[1,1] = m2

j = 0
for k in k_array:
    ## バネ定数の行列Kを定義 ##
    K = np.zeros((N, N), dtype = np.complex128)
    K[0,0] = k1 + k2
    K[1,1] = k1 + k2
    K[0,1] = -k1 -k2*np.exp(-1j*k)
    K[1,0] = -k1 -k2*np.exp(1j*k)
    
    ## 固有値方程式を解く ##
    M_inv = np.linalg.inv(M)  # Mの逆行列
    eigen_value, eigen_vector = np.linalg.eig(np.dot(M_inv, K))  # 固有値方程式を解く
    eigen_id = np.argsort(eigen_value)  # 固有値が小さい順にソートしたindexを取得
    print(eigen_id)
    eigen_values[j,:] = eigen_value[eigen_id]  # ソートした固有値、角周波数の２乗に対応
    eigen_vectors[j,:] = eigen_vector.T[eigen_id]   # ソートした固有ベクトル、複素数表示した変位に対応
    omega_square = eigen_values.real
    j += 1


### Plotlyによる可視化 ###
## 散布図プロット用の配列の準備 ##
x_scatter_array = np.ravel(k_array.reshape(k_num,1) * np.ones((k_num, m_num)))
omega_square_array = np.ravel(omega_square)

## mode表示用のトレースの指定 ##
scatter_trace = go.Scatter(x=x_scatter_array,
                           y=np.sqrt(omega_square_array),
                           mode="markers",
                           marker={"size": 10, "colorscale": "Bluered", "cmin": 0, "cmax": 1,},
                           name="mode",
                          )

frame_num = 15    # アニメーションのフレーム数
shape_period = 10  #単位格子何個分を表示するか

## グラフ表示
fig = go.FigureWidget(make_subplots(rows=2, cols=1, row_heights=[0.7,0.3],))
fig.update_layout(width=800, height=800, title="two atoms in an unit cell, $M_1>M_2, K_1=K_2$")
fig.add_trace(scatter_trace, row=1, col=1,)
fig.update_xaxes(title="wave number", row=1, col=1,)
fig.update_yaxes(title="angular frequency", row=1, col=1,)
fig.update_xaxes(tickvals=[-np.pi, -np.pi/2, 0, np.pi/2, np.pi], ticktext=[r"$-\pi$",r"$-\pi/2a$",r"$0$",r"$\pi/2a$",r"$\pi/a$"], row=1, col=1)
fig.update_xaxes(range=[0, 2*shape_period], visible=False, row=2, col=1,)
fig.update_yaxes(range=[-1.2,1.2], tickvals=[], ticktext=[], row=2, col=1)

fig.data[0].marker.color = [0]*(k_num*m_num)
fig.data[0].marker.size  = [10]*(k_num*m_num)
fig.data[0].marker.line.width = 0

for t in range(frame_num):    # 変位分布の初期プロットをフレーム数分準備
    fig.add_trace(go.Scatter(visible=False,
                             x=np.arange(m_num*shape_period+1),
                             y=np.zeros(m_num*shape_period+2),
                             mode = "lines+markers",
                             marker = {"size":20,"color":"gray"},
                             line={"width":5},
                             name="displacement",),
                  row=2, col=1)
fig.data[1].visible = True

steps = []    # 各スライダーでどのグラフを表示させ、どれを非表示にするか
for i in range(frame_num):
    step = {"method":"update",
            "args":[{"visible": [False]*(frame_num+1)},],  # 全てのグラフを一旦非表示
            "label":"",}
    step["args"][0]["visible"][0] = True     # 上のグラフは常に表示
    step["args"][0]["visible"][i+1] = True   # i番目のグラフも表示、上のグラフも含めるためずれるので+1
    steps += [step]

sliders = [{"pad":{"t": 50},     # スライダーの設定
            "steps":steps,
            "tickwidth":0,
            "transition":{"duration":0},
           }]
fig.update_layout(sliders=sliders)
    
def update_point(trace, points, selector): # modeをクリックした場合の関数
    i = points.point_inds[0]
    c = [0]*len(trace.x)
    s = [10]*len(trace.x)
    c[i] = 1
    s[i] = 20
    with fig.batch_update():
        fig.data[0].marker.color = c
        fig.data[0].marker.size = s
    for t in range(0, frame_num):     # 各フレームの計算
        fig.data[t+1].x = np.arange(m_num*shape_period+1)
        y0 = eigen_vectors[i//m_num, i%m_num, :]
        y_total = y0
        for sp in range(shape_period+1):     # 波数がずれた隣の単位格子の軌跡を繋げていく
            y_sp = y0*np.exp((sp+1)*1j*(trace.x)[i])
            y_total = np.concatenate((y_total,y_sp))
        fig.data[t+1].y = (y_total*np.exp(-2j*np.pi*t/(frame_num-1))).real
    fig.data[1].visible = True
    fig.update_layout({"sliders":{"active":0}})

fig.data[0].on_click(update_point)

display(fig)

[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]


FigureWidget({
    'data': [{'marker': {'cmax': 1,
                         'cmin': 0,
                       …

In [4]:
# 周期境界、Plotly版、sliderを使ってmodeをアニメーション
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

## パラメータ設定 ##　格子定数は1に設定
m0 = 1            # 基準の質量
k0 = 1            # 基準のバネ定数
m_num = 2       # 単位格子内の質点の数
k_num = 33        # 波数を変化させる数

k_array = np.linspace(-1, 1, k_num)*np.pi  # 波数の配列
omega_square = np.zeros((k_num, m_num))   # 周波数の２乗の配列を準備
eigen_values = np.zeros((k_num, m_num), dtype = np.complex128)    # 固有値の配列を準備、複素数
eigen_vectors = np.zeros((k_num, m_num, m_num), dtype = np.complex128)   # 固有ベクトルの配列を準備、複素数

delta_m = 0  # 単位格子内の質量
delta_k = 1/3    # 単位格子内のバネ定数
m1 = 1/(1 - delta_m)  # 質量1
m2 = 1/(1 + delta_m)  # 質量2
k1 = 1 + delta_k  # バネ1
k2 = 1 - delta_k  # バネ2
m_list = [m1,m2]  # 質量を並べたリスト
k_list = [k1,k2]  # バネ定数を並べたリスト、要素数は質量リストよりもひとつ多い
N = len(m_list)  # 質量の配列から行列サイズNを求める

## 質量の対角行列Mを定義 ##
M = np.zeros((N, N), dtype = np.complex128)
M[0,0] = m1
M[1,1] = m2

j = 0
for k in k_array:
    ## バネ定数の行列Kを定義 ##
    K = np.zeros((N, N), dtype = np.complex128)
    K[0,0] = k1 + k2
    K[1,1] = k1 + k2
    K[0,1] = -k1 -k2*np.exp(-1j*k)
    K[1,0] = -k1 -k2*np.exp(1j*k)
    
    ## 固有値方程式を解く ##
    M_inv = np.linalg.inv(M)  # Mの逆行列
    eigen_value, eigen_vector = np.linalg.eig(np.dot(M_inv, K))  # 固有値方程式を解く
    eigen_id = np.argsort(eigen_value)  # 固有値が小さい順にソートしたindexを取得
    print(eigen_id)
    print(eigen_value[eigen_id])
    eigen_values[j,:] = eigen_value[eigen_id]  # ソートした固有値、角周波数の２乗に対応
    eigen_vectors[j,:] = eigen_vector.T[eigen_id]   # ソートした固有ベクトル、複素数表示した変位に対応
    omega_square = eigen_values.real
    j += 1


### Plotlyによる可視化 ###
## 散布図プロット用の配列の準備 ##
x_scatter_array = np.ravel(k_array.reshape(k_num,1) * np.ones((k_num, m_num)))
omega_square_array = np.ravel(omega_square)

## mode表示用のトレースの指定 ##
scatter_trace = go.Scatter(x=x_scatter_array,
                           y=np.sqrt(omega_square_array),
                           mode="markers",
                           marker={"size": 10, "colorscale": "Bluered", "cmin": 0, "cmax": 1,},
                           name="mode",
                          )

frame_num = 15    # アニメーションのフレーム数
shape_period = 10  #単位格子何個分を表示するか

## グラフ表示
fig = go.FigureWidget(make_subplots(rows=2, cols=1, row_heights=[0.7,0.3],))
fig.update_layout(width=800, height=800, title="two atoms in an unit cell, $K_1>K_2, M_1=M_2$")
fig.add_trace(scatter_trace, row=1, col=1,)
fig.update_xaxes(title="wave number", row=1, col=1,)
fig.update_yaxes(title="angular frequency", row=1, col=1,)
fig.update_xaxes(tickvals=[-np.pi, -np.pi/2, 0, np.pi/2, np.pi], ticktext=[r"$-\pi$",r"$-\pi/2a$",r"$0$",r"$\pi/2a$",r"$\pi/a$"], row=1, col=1)
fig.update_xaxes(range=[0, 2*shape_period], visible=False, row=2, col=1,)
fig.update_yaxes(range=[-1.2,1.2], tickvals=[], ticktext=[], row=2, col=1)

fig.data[0].marker.color = [0]*(k_num*m_num)
fig.data[0].marker.size  = [10]*(k_num*m_num)
fig.data[0].marker.line.width = 0

for t in range(frame_num):    # 変位分布の初期プロットをフレーム数分準備
    fig.add_trace(go.Scatter(visible=False,
                             x=np.arange(m_num*shape_period+1),
                             y=np.zeros(m_num*shape_period+2),
                             mode = "lines+markers",
                             marker = {"size":20,"color":"gray"},
                             line={"width":5},
                             name="displacement",),
                  row=2, col=1)
fig.data[1].visible = True

steps = []    # 各スライダーでどのグラフを表示させ、どれを非表示にするか
for i in range(frame_num):
    step = {"method":"update",
            "args":[{"visible": [False]*(frame_num+1)},],  # 全てのグラフを一旦非表示
            "label":"",}
    step["args"][0]["visible"][0] = True     # 上のグラフは常に表示
    step["args"][0]["visible"][i+1] = True   # i番目のグラフも表示、上のグラフも含めるためずれるので+1
    steps += [step]

sliders = [{"pad":{"t": 50},     # スライダーの設定
            "steps":steps,
            "tickwidth":0,
            "transition":{"duration":0},
           }]
fig.update_layout(sliders=sliders)
    
def update_point(trace, points, selector): # modeをクリックした場合の関数
    i = points.point_inds[0]
    c = [0]*len(trace.x)
    s = [10]*len(trace.x)
    c[i] = 1
    s[i] = 20
    with fig.batch_update():
        fig.data[0].marker.color = c
        fig.data[0].marker.size = s
    for t in range(0, frame_num):     # 各フレームの計算
        fig.data[t+1].x = np.arange(m_num*shape_period+1)
        y0 = eigen_vectors[i//m_num, i%m_num, :]
        y_total = y0
        for sp in range(shape_period+1):     # 波数がずれた隣の単位格子の軌跡を繋げていく
            y_sp = y0*np.exp((sp+1)*1j*(trace.x)[i])
            y_total = np.concatenate((y_total,y_sp))
        fig.data[t+1].y = (y_total*np.exp(-2j*np.pi*t/(frame_num-1))).real
    fig.data[1].visible = True
    fig.update_layout({"sliders":{"active":0}})

fig.data[0].on_click(update_point)

display(fig)

[1 0]
[1.33333333+6.16297582e-33j 2.66666667-6.16297582e-33j]
[0 1]
[1.30818793+0.j 2.69181207+0.j]
[1 0]
[1.23857389-1.38777878e-17j 2.76142611+1.38777878e-17j]
[0 1]
[1.13741434-5.55111512e-17j 2.86258566-5.55111512e-17j]
[1 0]
[1.01758283+5.55111512e-17j 2.98241717+5.55111512e-17j]
[1 0]
[0.88890063+8.32667268e-17j 3.11109937+2.77555756e-17j]
[1 0]
[0.7582689-5.55111512e-17j 3.2417311-5.55111512e-17j]
[1 0]
[0.63054939-5.55111512e-17j 3.36945061+5.55111512e-17j]
[1 0]
[0.50928802+0.00000000e+00j 3.49071198-1.11022302e-16j]
[1 0]
[0.39717454+0.j 3.60282546+0.j]
[1 0]
[0.29631331+5.55111512e-17j 3.70368669-1.66533454e-16j]
[1 0]
[0.20837988-1.11022302e-16j 3.79162012+3.69778549e-32j]
[0 1]
[0.13471157-1.11022302e-16j 3.86528843-1.74315280e-32j]
[0 1]
[0.07636011+0.00000000e+00j 3.92363989+5.55111512e-17j]
[1 0]
[0.0341224-2.77555756e-17j 3.9658776+2.77555756e-17j]
[0 1]
[0.00855819+6.16297582e-33j 3.99144181-2.77555756e-17j]
[1 0]
[2.88815012e-32+0.j 4.00000000e+00+0.j]
[0 1]
[0.00855

FigureWidget({
    'data': [{'marker': {'cmax': 1,
                         'cmin': 0,
                       …